[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dina-lab3D/NanoNet/blob/main/NanoNet.ipynb)


#NanoNet
NanoNet is a novel deep learning-based end-to-end modeling tool that given a 
sequence directly produces the 3D coordinates of the Cβ atoms of the entire VH domain.
It can be used in order to predict structures of nanobodies, VH regions of mAbs and VB regions of TCRs.

The source code and the trained model can be found [here](https://github.com/dina-lab3D/NanoNet)


<br>
<img src=https://drive.google.com/uc?id=1DdACpv5loaOnKbrIIlJSygUUmt9dRUut width="2000">


**Note**: If you are predicting many structures in a single run, it is recomanded that you use the option 'write_into_single_pdb_file'. this will write all the predicted structures into a single PDB file and reduce running time and space.


<strong>For Citation use: </strong> [NanoNet paper](https://www.frontiersin.org/articles/10.3389/fimmu.2022.958584/full)


In [ ]:
#@title Input Nb sequence/Nb fasta file (creates a Nb structure for each entry in the fasta file)
from google.colab import files
import re
import os
from IPython.display import clear_output


protein = 'Nb/mAb heavy chain' #@param ["Nb/mAb heavy chain", "TCR VB"]
input_type = 'Sequence (String)' #@param ["Sequence (String)", "Fasta file"]
input_nb = 'QLVETGGGLVQSGGSLRLSCAASGFTLDNYNIGWFRQAPGKEYGGVSCISSSDGSTYYADSVKGRFTISRDNAKNTVYLQMNNLKPEDTDVYYCAATKYGSSCPIRPYDYWGQGTQVT' #@param {type:"string"}
output_dir = 'NanoNetResults' #@param {type:"string"}

# remove whitespaces
output_dir = "".join(output_dir.split())
output_dir = re.sub(r'\W+', '', output_dir)


# get NanoNet trained model path
tcr_modeling = False if protein == 'Nb/mAb heavy chain' else True

#@markdown ---
#@markdown ### Advanced settings
write_into_single_pdb_file = False #@param {type:"boolean"}
reconsrtuct_side_chains_using_modeller = False #@param {type:"boolean"}
modeller_license_key = '' #@param {type:"string"}
visualize_results = True #@param {type:"boolean"}
# reconsrtuct_side_chains_using_scwrl = "" #@param {type:"string"}
##@markdown  (insert Scwrl4 executable path)
# reconsrtuct_side_chains_using_modeller = False
reconsrtuct_side_chains_using_scwrl = False

#@markdown ---
#@markdown ### Saving options
save_to_google_drive = False #@param {type:"boolean"}
#@markdown ---
#@markdown  To run NanoNet hit `Runtime` -> `Run all`

if reconsrtuct_side_chains_using_modeller and write_into_single_pdb_file:
  raise ValueError("Can't reconstruct side chains with single_file option select only one option from 'write_into_single_pdb_file' and 'reconsrtuct_side_chains_using_modeller'")
if reconsrtuct_side_chains_using_modeller and modeller_license_key == '':
  raise ValueError("Please insert a valid license key!, you can get one from here: https://salilab.org/modeller/registration.html")

if input_type == 'Sequence (String)':  
  # remove whitespaces
  input_nb = "".join(input_nb.split())
  input_nb = re.sub(r'[^a-zA-Z]','', input_nb).upper()
  with open("nb_fasta.fa", "w") as fa_file:
    fa_file.write("> model\n")
    fa_file.write("{}\n".format(input_nb))
  input_nb = "nb_fasta.fa"

if not os.path.exists(input_nb):
  raise ValueError("can't find fasta file {}.".format(input_nb))

if save_to_google_drive == True:
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Saving results into Drive")


In [ ]:
#@title Clone NanoNet trained model 


%%bash

if [ ! -f NanoNetReady ]; then
  # install dependencies
  pip -q install biopython
  pip install py3Dmol

  # download model
  if [ ! -d "NanoNet/" ]; then
    git clone https://github.com/dina-lab3D/NanoNet --quiet
  fi
  touch NanoNetReady
fi





In [ ]:
#@title Download Modeller
#@markdown If 'reconsrtuct_side_chains_using_modeller' is set to false, you can skip this step.
if not os.path.exists("ModellerReady") and reconsrtuct_side_chains_using_modeller:
  #@markdown **You can get a license key for Modeller from** **[here](https://salilab.org/modeller/registration.html)** .
  # modeller_license_key = '' #@param {type:"string"}
  #MODELIRANJE
  !wget https://salilab.org/modeller/10.1/modeller-10.1.tar.gz
  !tar -zxf modeller-10.1.tar.gz
  print("MODELLER extraction completed")
  %cd modeller-10.1
  #And we prepare a file containing the minimal setup elements
  #For installing, including a license key
  with open('modeller_config', 'a') as f:
    f.write("3\n")
    f.write("/content/compiled/MODELLER\n")
  #ADD YOUR LICENSE KEY HERE!
    f.write(f"{modeller_license_key}\n")
  !./Install < modeller_config
  print("MODELLER set up completed")

  %cd /content/
  #Creating a symbolic link
  %cd modeller-10.1
  !ln -sf /content/compiled/MODELLER/bin/mod10.1 /usr/bin/
  %cd /content/
  #Checking if MODELLER works
  !mod10.1 | awk 'NR==1{if($1=="usage:") print "MODELLER succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

  with open("/content/compiled/MODELLER/modlib/modeller/config.py", "r") as file:
    lines = file.readlines()

  with open("/content/compiled/MODELLER/modlib/modeller/config.py", "w") as file:
    file.write(lines[0])
    file.write(f"license = '{modeller_license_key}'\n")
  with open("ModellerReady","w"):pass
  clear_output()


In [ ]:
#@title Predict Nb structure

import py3Dmol
os.chdir("/content/")
if "NanoNet" not in dir():
  from timeit import default_timer as timer
  import sys
  sys.path.insert(0, '/content/NanoNet/')
  from NanoNet import *

nanonet_dir_path = "/content/NanoNet"
nano_net_path = os.path.join(nanonet_dir_path, 'NanoNetTCR') if tcr_modeling else os.path.join(nanonet_dir_path, 'NanoNet')
error = False

if not os.path.exists(nano_net_path):
    print("Can't find trained NanoNet '{}', aborting.".format(nano_net_path), file=sys.stderr)
    error = True

if write_into_single_pdb_file and (reconsrtuct_side_chains_using_modeller or reconsrtuct_side_chains_using_scwrl):
    print("Can't reconstruct side chains with single_file option.",file=sys.stderr)
    error = True

if not error:

    flags = ""
    if tcr_modeling: flags+=" -t"
    if write_into_single_pdb_file: flags+=" -s"
    if reconsrtuct_side_chains_using_modeller: flags+=" -m"
    if reconsrtuct_side_chains_using_modeller:
      !/content/compiled/MODELLER/bin/modpy.sh python NanoNet/NanoNet.py  $input_nb -o $output_dir $flags
    else:
      !python NanoNet/NanoNet.py $input_nb -o $output_dir $flags
    # run_nanonet(input_nb, nano_net_path, write_into_single_pdb_file, output_dir, reconsrtuct_side_chains_using_modeller, reconsrtuct_side_chains_using_scwrl)


def plot_structure(nb_name, pdb_path):
    with open(pdb_path) as ifile:
      predicted = "".join([x for x in ifile])
    r,g,b = 0,0,255
    print(f"\033[38;2;{r};{g};{b}m {nb_name} Predicted model \033[38;2;255;255;255m")
    view = py3Dmol.view(width=500, height=500)
    view.addModelsAsFrames(predicted)
    view.setStyle({'model': 0}, {"cartoon": {'arrows':True, 'color': 'blue'}})
    view.zoomTo()
    view.show()



file_ending = "nanonet_backbone_cb.pdb" if not reconsrtuct_side_chains_using_modeller else "nanonet_full_relaxed.pdb"
if visualize_results and not write_into_single_pdb_file:
  print("Showing nanonet predicted structures")
  for seq, nb_name in seq_iterator(input_nb):
    plot_structure(nb_name, os.path.join(output_dir, f"{nb_name}_{file_ending}"))


In [ ]:
#@title Download results


!zip -FSr $output_dir".zip" $output_dir
files.download(f"{output_dir}.zip")

if save_to_google_drive == True and drive != None:
  uploaded = drive.CreateFile({'title': f"{output_dir}.zip"})
  uploaded.SetContentFile(f"{output_dir}.zip")
  uploaded.Upload()
  print(f"Uploaded {output_dir}.zip to Google Drive with ID {uploaded.get('id')}")